In [1]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

# Laser Shading By adjusting focus on the Z-axis

# Code:

In [2]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # [mm]
BlockLength = 2 * BlockHeight  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


89.0 x 178.0 x 19.0


In [3]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset(home=True)

if "Alarm" in cnc.status:
    print("Homing.")
    cnc.home()


Homing.


# Generate Program

Generate the G-code program that will test how the laser focus affects shading.

- Laser 'focused' at -14mm.
- Testing 17 positions between [-28, 0]
- Testing 3 different feed rates [100, 200, 300]
- Testing 5 different power settings between [100, 255]

In [4]:
# Define a gap between horizontal lines
vgap = 5  # [mm]
# Generate a list of horizontal line vertical coordinates
vs = np.arange(vgap, BlockHeight, vgap)
vs


array([ 5., 10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60., 65.,
       70., 75., 80., 85.])

In [5]:
# Define a horizontal distance to test lazerin.
hgap = 10  # [mm]
hs = np.arange(0, BlockLength, hgap)
hs


array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170.])

In [9]:
# Total travel range, starts at ztravel, ends at 0. "negative space"(?)
ztravel = -28
# Generate a number of z-axis test positions.
# Each z-height will have 10 mm marked at the given feed rate and power.

# Generate the full list of z-stops from the initial test.
zs_ = np.linspace(start=ztravel, stop=0, num=len(hs) - 1, endpoint=True)

zs = np.linspace(start=zs_[5], stop=zs_[13], num=len(hs) - 1, endpoint=True)


-5.25

In [8]:
-# Laser powers to test.
powers = np.linspace(255, 100, 5, endpoint=True, dtype=np.uint8)
powers


array([255, 216, 177, 138, 100], dtype=uint8)

In [9]:
def init2(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=500)
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    return program


In [12]:
feeds = [100, 300, 200]
programs = list()
for idx1, feed in enumerate(feeds):  # [mm/min]
    for idx2, power in enumerate(powers):

        v_idx = idx1 * len(feeds) + idx2
        Y = vs[v_idx]
        for idx3, z in enumerate(zs):
            x0 = hs[idx3]
            xf = hs[idx3 + 1]

            program = init2(machine=cnc)
            program.G0(Z=z, Y=Y, X=x0)
            program.M4(S=power)
            program.G1(X=xf, Y=Y, Z=z, F=feed)
            program.M5()  # Laser. Off.
            programs.append(program)


In [13]:
programs


[<GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[cmds=8],
 <GCode>[c

In [14]:
programs[17]


<GCode>[cmds=8]

In [16]:
zs[0]


-28.0

In [17]:
zs[5]


-19.25

In [19]:
zs[13]


-5.25

In [ ]:
for idx, program in enumerate(programs):
    print("{}:".format(idx))
    program.machine = cnc
    program.run()


0:
1:
2:
3:
4:
5:
6:
7:
8:
9:
10:
11:
12:
13:
14:
15:
16:
^C
17:
^C
18:


In [ ]:
cnc.status
